In [1]:
import asyncio
import json
import os
from typing import List, Dict, Any
from enum import Enum

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from graphiti_core import Graphiti


In [2]:
# Simple context quality assessment
class ContextQuality(Enum):
    INSUFFICIENT = "insufficient"
    SUFFICIENT = "sufficient"
    EXCESSIVE = "excessive"

In [3]:
class SimpleContextAgent:
    def __init__(self):
        # Setup
        self.llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        self.client = Graphiti(
            os.environ.get('NEO4J_URI', 'bolt://localhost:7687'),
            os.environ.get('NEO4J_USER', 'neo4j'),
            os.environ.get('NEO4J_PASSWORD', 'password')
        )
        
    async def get_context(self, query: str, max_results: int = None) -> tuple[List[str], List[Dict]]:
        """Get context from knowledge base and return both facts and detailed results"""
        try:
            # Dynamic max_results based on query if not specified
            if max_results is None:
                max_results = await self.determine_initial_fetch_size(query)
            
            print(f"\n🔍 SEARCHING GRAPH DATABASE:")
            print(f"   Query: '{query}'")
            print(f"   Dynamic Max Results: {max_results}")
            
            results = await self.client.search(query, num_results=max_results)
            
            # Extract facts and detailed information
            facts = []
            detailed_results = []
            
            print(f"\n📊 GRAPH DATABASE RESULTS ({len(results)} items found):")
            print("-" * 60)
            
            for i, edge in enumerate(results, 1):
                fact = edge.fact
                facts.append(fact)
                
                # Create detailed result info
                result_info = {
                    'index': i,
                    'fact': fact,
                    'confidence': getattr(edge, 'confidence', 'N/A'),
                    'source_node': getattr(edge, 'source_node', 'N/A'),
                    'target_node': getattr(edge, 'target_node', 'N/A'),
                    'edge_type': getattr(edge, 'edge_type', 'N/A'),
                    'metadata': getattr(edge, 'metadata', {})
                }
                detailed_results.append(result_info)
                
                print(f"{i:2d}. Fact: {fact}")
                print(f"    Confidence: {result_info['confidence']}")
                print(f"    Source: {result_info['source_node']}")
                print(f"    Target: {result_info['target_node']}")
                print(f"    Type: {result_info['edge_type']}")
                if result_info['metadata']:
                    print(f"    Metadata: {result_info['metadata']}")
                print()
            
            if not results:
                print("   ❌ No results found in graph database")
            
            return facts, detailed_results
            
        except Exception as e:
            print(f"   ❌ Error searching graph database: {str(e)}")
            return [], []
    
    async def determine_initial_fetch_size(self, query: str) -> int:
        """Dynamically determine how many facts to fetch initially based on query complexity"""
        
        analysis_prompt = f"""
        Analyze this query and determine how many facts should be initially fetched from a knowledge base:
        
        Query: "{query}"
        
        Consider:
        - Simple factual questions: 3-5 facts
        - Complex multi-part questions: 8-12 facts  
        - Broad exploratory questions: 10-15 facts
        - Comparison questions: 6-10 facts
        - Technical/detailed questions: 8-12 facts
        
        Respond with ONLY a number between 3 and 15.
        """
        
        try:
            print(f"   🧠 Analyzing query complexity...")
            response = await self.llm.ainvoke([HumanMessage(content=analysis_prompt)])
            initial_size = int(response.content.strip())
            
            # Clamp to reasonable bounds
            initial_size = max(3, min(15, initial_size))
            print(f"   📊 Determined initial fetch size: {initial_size}")
            return initial_size
            
        except Exception as e:
            print(f"   ❌ Error determining fetch size: {str(e)}")
            print(f"   🔄 Using default size: 8")
            return 8  # Default fallback
    
    async def assess_context_quality(self, query: str, context: List[str], target_answer_depth: str = "auto") -> Dict[str, Any]:
        """Check if context is sufficient, insufficient, or excessive"""
        
        # Determine target depth if auto
        if target_answer_depth == "auto":
            target_answer_depth = await self.determine_answer_depth_needed(query)
        
        print(f"\n🔍 ASSESSING CONTEXT QUALITY:")
        print(f"   Query: '{query}'")
        print(f"   Context items: {len(context)}")
        print(f"   Target answer depth: {target_answer_depth}")
        
        assessment_prompt = f"""
        Query: {query}
        Target Answer Depth: {target_answer_depth}
        
        Context Facts ({len(context)} items):
        {chr(10).join([f"- {fact}" for fact in context]) if context else "No context provided"}
        
        Assess if this context is sufficient for a {target_answer_depth} answer to this query.
        
        Respond in JSON:
        {{
            "quality": "insufficient|sufficient|excessive",
            "confidence": 0.0-1.0,
            "reasoning": "brief explanation",
            "can_answer": true/false,
            "missing_info": "what key information is missing (if any)",
            "relevance_score": 0.0-1.0,
            "recommended_context_size": number_of_facts_needed,
            "current_coverage": 0.0-1.0
        }}
        
        Guidelines based on target depth:
        - brief: 2-4 highly relevant facts sufficient
        - moderate: 4-8 relevant facts needed  
        - comprehensive: 8-15+ facts for thorough coverage
        - insufficient: Missing key information needed
        - sufficient: Has enough relevant information for target depth
        - excessive: Too much irrelevant information that clutters
        """
        
        try:
            print("   📤 Sending assessment prompt to LLM...")
            response = await self.llm.ainvoke([HumanMessage(content=assessment_prompt)])
            assessment = json.loads(response.content)
            
            print(f"   📥 LLM Assessment Result:")
            print(f"      Quality: {assessment.get('quality', 'unknown')}")
            print(f"      Confidence: {assessment.get('confidence', 0):.2f}")
            print(f"      Can Answer: {assessment.get('can_answer', False)}")
            print(f"      Reasoning: {assessment.get('reasoning', 'N/A')}")
            print(f"      Missing Info: {assessment.get('missing_info', 'N/A')}")
            print(f"      Relevance Score: {assessment.get('relevance_score', 0):.2f}")
            print(f"      Recommended Size: {assessment.get('recommended_context_size', 'N/A')}")
            print(f"      Current Coverage: {assessment.get('current_coverage', 0):.2f}")
            
            return assessment
            
        except Exception as e:
            print(f"   ❌ Error in LLM assessment: {str(e)}")
            # Fallback simple assessment
            fallback = {
                "quality": "insufficient" if not context else ("excessive" if len(context) > 12 else "sufficient"),
                "confidence": 1.0 if not context else (0.7 if len(context) > 12 else 0.8),
                "reasoning": "No context" if not context else ("Too many facts" if len(context) > 12 else "Reasonable amount"),
                "can_answer": bool(context),
                "missing_info": "No context available" if not context else "N/A",
                "relevance_score": 0.0 if not context else 0.7,
                "recommended_context_size": 8 if not context else len(context),
                "current_coverage": 0.0 if not context else 0.7
            }
            print(f"   🔄 Using fallback assessment: {fallback}")
            return fallback
    
    async def determine_answer_depth_needed(self, query: str) -> str:
        """Determine if query needs brief, moderate, or comprehensive answer"""
        
        depth_prompt = f"""
        Analyze this query and determine what depth of answer is needed:
        
        Query: "{query}"
        
        Choose ONE:
        - brief: Simple factual answer, 1-2 sentences
        - moderate: Explanation with some detail, 1-2 paragraphs  
        - comprehensive: Detailed analysis with examples, multiple paragraphs
        
        Respond with ONLY the word: brief, moderate, or comprehensive
        """
        
        try:
            response = await self.llm.ainvoke([HumanMessage(content=depth_prompt)])
            depth = response.content.strip().lower()
            if depth in ['brief', 'moderate', 'comprehensive']:
                return depth
            return 'moderate'  # Default
        except:
            return 'moderate'  # Default fallback
    
    async def optimize_context(self, query: str, initial_context: List[str], detailed_results: List[Dict]) -> tuple[List[str], List[Dict]]:
        """Get the right amount of context - not too much, not too little"""
        
        print(f"\n🔧 OPTIMIZING CONTEXT:")
        print(f"   Initial context items: {len(initial_context)}")
        
        if not initial_context:
            print("   📈 No initial context - trying broader search...")
            # Dynamic broader search based on assessment
            broader_size = max(15, len(initial_context) * 2)
            broader_context, broader_details = await self.get_context(query, max_results=broader_size)
            
            # Use smart selection instead of just first N
            optimized_context, optimized_details = await self.smart_select_context(
                query, broader_context, broader_details, target_size="auto"
            )
            print(f"   ✅ Broader search returned {len(broader_context)} items, intelligently selected {len(optimized_context)}")
            return optimized_context, optimized_details
        
        # Smart context selection
        print("   🎯 Using intelligent context selection...")
        return await self.smart_select_context(query, initial_context, detailed_results, target_size="auto")
    
    async def smart_select_context(self, query: str, context: List[str], details: List[Dict], target_size: str = "auto") -> tuple[List[str], List[Dict]]:
        """Intelligently select the optimal number and combination of context facts"""
        
        if target_size == "auto":
            # Determine optimal size based on query and available context
            target_size = await self.determine_optimal_context_size(query, len(context))
        
        print(f"   🎯 Smart selecting from {len(context)} facts, target: {target_size}")
        
        if len(context) <= target_size:
            print(f"   ✅ Available context ({len(context)}) <= target ({target_size}), using all")
            return context, details
        
        # Use LLM to intelligently select best facts
        selection_prompt = f"""
        Query: {query}
        Target number of facts: {target_size}
        
        Available Context:
        {chr(10).join([f"{i+1}. {fact}" for i, fact in enumerate(context)])}
        
        Select the {target_size} most relevant and complementary facts that together provide the best coverage for answering this query.
        
        Consider:
        - Direct relevance to the query
        - Complementary information (avoid redundancy)  
        - Comprehensive coverage of the topic
        - Quality and specificity of information
        
        Return ONLY the numbers separated by commas (e.g., "1,3,5,7,9"):
        """
        
        try:
            print("   📤 Sending smart selection prompt to LLM...")
            response = await self.llm.ainvoke([HumanMessage(content=selection_prompt)])
            selected_nums = [int(n.strip()) for n in response.content.split(',') if n.strip().isdigit()]
            
            # Validate selection
            selected_nums = [n for n in selected_nums if 0 < n <= len(context)]
            if not selected_nums:
                # Fallback to first target_size items
                selected_nums = list(range(1, min(target_size + 1, len(context) + 1)))
            
            print(f"   📥 LLM selected indices: {selected_nums}")
            
            # Apply selection
            selected_context = [context[i-1] for i in selected_nums]
            selected_details = [details[i-1] for i in selected_nums]
            
            print(f"   ✅ Selected {len(selected_context)} optimal facts")
            
            # Print selected items
            for i, fact in enumerate(selected_context, 1):
                print(f"   {i}. Selected: {fact[:80]}...")
            
            return selected_context, selected_details
            
        except Exception as e:
            print(f"   ❌ Error in smart selection: {str(e)}")
            print(f"   🔄 Using fallback: first {target_size} items")
            return context[:target_size], details[:target_size]
    
    async def determine_optimal_context_size(self, query: str, available_count: int) -> int:
        """Determine optimal number of context facts needed for this specific query"""
        
        optimization_prompt = f"""
        Query: "{query}"
        Available facts: {available_count}
        
        Determine the optimal number of facts needed to answer this query well.
        
        Consider:
        - Query complexity and scope
        - Need for comprehensive vs focused answer
        - Risk of information overload vs insufficient detail
        
        Respond with ONLY a number between 2 and {min(available_count, 15)}.
        """
        
        try:
            response = await self.llm.ainvoke([HumanMessage(content=optimization_prompt)])
            optimal_size = int(response.content.strip())
            
            # Clamp to reasonable bounds
            optimal_size = max(2, min(available_count, optimal_size))
            print(f"   📊 Optimal context size determined: {optimal_size}")
            return optimal_size
            
        except Exception as e:
            print(f"   ❌ Error determining optimal size: {str(e)}")
            # Smart fallback based on available count
            fallback_size = min(8, max(3, available_count // 2))
            print(f"   🔄 Using smart fallback size: {fallback_size}")
            return fallback_size
    
    async def answer_with_context(self, query: str) -> Dict[str, Any]:
        """Main method: Get context, assess quality, and answer"""
        
        print(f"\n{'='*80}")
        print(f"🚀 PROCESSING QUERY: '{query}'")
        print(f"{'='*80}")
        
        # Step 1: Get initial context
        print(f"\n📝 STEP 1: INITIAL CONTEXT RETRIEVAL")
        initial_context, initial_details = await self.get_context(query)
        
        # Step 2: Assess context quality
        print(f"\n📝 STEP 2: INITIAL CONTEXT ASSESSMENT")
        assessment = await self.assess_context_quality(query, initial_context)
        
        # Step 3: Optimize context if needed
        print(f"\n📝 STEP 3: DYNAMIC CONTEXT OPTIMIZATION")
        if assessment["quality"] == "insufficient":
            print("   🔄 Context insufficient - expanding search dynamically...")
            
            # Determine how much more context we need
            recommended_size = assessment.get("recommended_context_size", len(initial_context) * 2)
            expanded_context, expanded_details = await self.get_context(query, max_results=recommended_size)
            
            optimized_context, optimized_details = await self.smart_select_context(
                query, expanded_context, expanded_details, target_size="auto"
            )
        elif assessment["quality"] == "excessive":
            print("   🔄 Context excessive - intelligently reducing...")
            
            # Determine optimal reduced size
            target_size = max(3, assessment.get("recommended_context_size", len(initial_context) // 2))
            optimized_context, optimized_details = await self.smart_select_context(
                query, initial_context, initial_details, target_size=target_size
            )
        else:
            print("   ✅ Context quality sufficient - minor optimization...")
            # Even for sufficient context, do light optimization
            optimized_context, optimized_details = await self.smart_select_context(
                query, initial_context, initial_details, target_size="auto"
            )
        
        # Step 4: Final assessment
        print(f"\n📝 STEP 4: FINAL CONTEXT ASSESSMENT")
        final_assessment = await self.assess_context_quality(query, optimized_context)
        
        # Step 5: Generate answer
        print(f"\n📝 STEP 5: ANSWER GENERATION")
        if final_assessment["can_answer"]:
            print("   ✅ Sufficient context available - generating answer...")
            
            # Print final context being used
            print(f"\n📋 FINAL CONTEXT BEING USED FOR ANSWER:")
            for i, (fact, detail) in enumerate(zip(optimized_context, optimized_details), 1):
                print(f"   {i}. {fact}")
                print(f"      └─ Confidence: {detail.get('confidence', 'N/A')}")
                print(f"      └─ Source: {detail.get('source_node', 'N/A')} → {detail.get('target_node', 'N/A')}")
            
            answer_prompt = f"""
            Query: {query}
            
            Context:
            {chr(10).join([f"- {fact}" for fact in optimized_context])}
            
            Provide a helpful answer based on the context above. Be specific about which facts you're using.
            """
            
            print("   📤 Sending answer generation prompt to LLM...")
            answer_response = await self.llm.ainvoke([HumanMessage(content=answer_prompt)])
            answer = answer_response.content
            print(f"   📥 Answer generated ({len(answer)} characters)")
        else:
            print("   ❌ Insufficient context - cannot provide reliable answer")
            answer = "I don't have enough relevant information to answer your question properly."
        
        # Compile final result
        result = {
            "query": query,
            "initial_context_count": len(initial_context),
            "final_context_count": len(optimized_context),
            "context_quality": final_assessment["quality"],
            "confidence": final_assessment["confidence"],
            "reasoning": final_assessment["reasoning"],
            "context_facts": optimized_context,
            "detailed_context": optimized_details,
            "initial_assessment": assessment,
            "final_assessment": final_assessment,
            "answer": answer
        }
        
        print(f"\n🎉 PROCESSING COMPLETE")
        print(f"   Final Context Quality: {result['context_quality']}")
        print(f"   Confidence: {result['confidence']:.2f}")
        print(f"   Context Items Used: {result['final_context_count']}")
        
        return result

In [4]:
# Tool for external use
@tool
async def smart_search(query: str) -> str:
    """Search with intelligent context optimization"""
    agent = SimpleContextAgent()
    result = await agent.answer_with_context(query)
    
    return f"""
    Context Quality: {result['context_quality']} (confidence: {result['confidence']:.2f})
    Context Used: {result['final_context_count']} facts
    Reasoning: {result['reasoning']}
    
    Answer: {result['answer']}
    """

In [5]:
agent = SimpleContextAgent()

# Test with different types of queries
queries = [
    "What are the best running shoes?",
    "Tell me about customer service policies",
    "How do I return a product?"
]

for query in queries:
    result = await agent.answer_with_context(query)
    
    print(f"\n{'🎯 FINAL SUMMARY':=^80}")
    print(f"Query: {result['query']}")
    print(f"Initial Context: {result['initial_context_count']} facts")
    print(f"Final Context: {result['final_context_count']} facts")
    print(f"Quality: {result['context_quality']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reasoning: {result['reasoning']}")
    print(f"\nDetailed Context Used:")
    for detail in result['detailed_context']:
        print(f"  • {detail['fact'][:100]}...")
    print(f"\nFinal Answer:\n{result['answer']}")
    print(f"{'='*80}")



🚀 PROCESSING QUERY: 'What are the best running shoes?'

📝 STEP 1: INITIAL CONTEXT RETRIEVAL
   🧠 Analyzing query complexity...
   📊 Determined initial fetch size: 10

🔍 SEARCHING GRAPH DATABASE:
   Query: 'What are the best running shoes?'
   Dynamic Max Results: 10

📊 GRAPH DATABASE RESULTS (10 items found):
------------------------------------------------------------
 1. Fact: Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 2. Fact: Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 3. Fact: TinyBirds Wool Runners - Little Kids - Natural Black (Blizzard Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 4. Fact: TinyBirds Wool Runners - Little Kids - Natural Black (Blizzard Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target